In [ ]:
## Data Initializing

In [1]:
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3.7 get-pip.py
!pip install scanf
!pip install numba

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1734k  100 1734k    0     0   309k      0  0:00:05  0:00:05 --:--:--  398k
     |████████████████████████████████| 1.4MB 833kB/s eta 0:00:01
     |████████████████████████████████| 3.6MB 851kB/s eta 0:00:01
     |████████████████████████████████| 20.2MB 5.9MB/s eta 0:00:01     |██████████████████▌             | 11.6MB 6.4MB/s eta 0:00:02


In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [2]:
net_name = 'Anaheim_net.tntp'
trips_name = 'Anaheim_trips.tntp'

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns_order = np.array([0, 1, 2, 4]))
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

model = md.Model(graph_data, graph_correspondences, 
                    total_od_flow, mu = 0.25, rho = 0.15)

graph_data['graph_table'].head()

,Init node,Term node,Capacity,Free Flow Time
0,1,117,9000.0,1.090459
1,2,87,9000.0,1.090459
2,3,74,9000.0,1.090459
3,4,233,9000.0,1.090459
4,5,165,9000.0,1.090459


### For Beckman's model:

### Universal similar triangles method

In [4]:
model.mu = 0.25
max_iter = 10000
for i, eps_abs in enumerate(np.logspace(1,3,5)[0:2]):
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ustf', solver_kwargs = solver_kwargs, verbose = True) #verbose = False
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['Free Flow Time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['Capacity']) - 1, end = '\n\n')
    
    result['eps_abs'] = eps_abs
    #with open('anaheim_result_' + 'ustf' + '_#' + str(i) + '_max_iter_' + str(max_iter) + 
    #          '_Beckman.pickle', 'wb') as f:
    #    pickle.dump(result, f)
    #these results in 'results' folder

eps_abs = 10.0
Oracles created...
Universal similar triangles function...
Primal_init = 1.26452e+06
Dual_init = -1.17208e+06
Duality_gap_init = 92434.2

Iterations number: 1
Inner iterations number: 1
Primal_func_value = 1.26452e+06
Dual_func_value = -1.17208e+06
Duality_gap = 92434.2
Duality_gap / Duality_gap_init = 1

Iterations number: 100
Inner iterations number: 1
Primal_func_value = 1.2077e+06
Dual_func_value = -1.20161e+06
Duality_gap = 6095.51
Duality_gap / Duality_gap_init = 0.0659443

Iterations number: 200
Inner iterations number: 1
Primal_func_value = 1.2074e+06
Dual_func_value = -1.20237e+06
Duality_gap = 5023.31
Duality_gap / Duality_gap_init = 0.0543448

Iterations number: 300
Inner iterations number: 2
Primal_func_value = 1.20718e+06
Dual_func_value = -1.20283e+06
Duality_gap = 4347.75
Duality_gap / Duality_gap_init = 0.0470362

Iterations number: 400
Inner iterations number: 3
Primal_func_value = 1.207e+06
Dual_func_value = -1.20329e+06
Duality_gap = 3711.81
Duality_ga

KeyboardInterrupt: 

### Frank-Wolfe algorithm

In [7]:
model.mu = 0.25
max_iter = 10000

print('Frank-Wolfe without stopping criteria')
solver_kwargs = {'max_iter': max_iter}
tic = time.time()
result = model.find_equilibrium(solver_name = 'fwa', solver_kwargs = solver_kwargs, verbose = False)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))

print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['Free Flow Time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['Capacity']) - 1, end = '\n\n')

with open('anaheim_result_' + 'fwa' + '_max_iter_' + str(max_iter) + 
          '_Beckman.pickle', 'wb') as f:
    pickle.dump(result, f)

Frank-Wolfe without stopping criteria
L_init =  1.0
Elapsed time: 1185 sec
Time ratio = 2.9107237337629295
Flow excess = 0.8891944444444517



# Our experiments

In [5]:
model.mu = 0.25
max_iter = 10000
# for i, eps_abs in enumerate(np.logspace(1,3,5)[0:2]):
eps_abs = 1e-3
print('eps_abs =', eps_abs)
solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter}
tic = time.time()
result = model.find_equilibrium(solver_name = 'updag', solver_kwargs = solver_kwargs, verbose = True) #verbose = False
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
    
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['Free Flow Time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['Capacity']) - 1, end = '\n\n')
    
result['eps_abs'] = eps_abs
    #with open('anaheim_result_' + 'ustf' + '_#' + str(i) + '_max_iter_' + str(max_iter) + 
    #          '_Beckman.pickle', 'wb') as f:
    #    pickle.dump(result, f)
    #these results in 'results' folder

eps_abs = 0.001
Oracles created...
Universal primal dual accelerated gradient descent...
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=0.5308036804199219
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06
Primal_init = 1.18319e+06
Dual_init = -1.15206e+06
Duality_gap_init = 31129.9

Iterations number: 1
Primal_func_value = 1.18319e+06
Dual_func_value = -1.15206e+06
Duality_gap = 31129.9
Duality_gap / Duality_gap_init = 1
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 2
Primal_func_value = 1.20789e+06
Dual_func_value = -1.16762e+06
Duality_gap = 40272.3
Duality_gap / Duality_gap_init = 1.29368
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 3
Primal_func_value = 1.21346e+06
Dual_func_value = -1.17546e+06
Duality_gap = 38000.3
Duality_gap / Duality_gap_init = 1.2207
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814

Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 36
Primal_func_value = 1.20753e+06
Dual_func_value = -1.20223e+06
Duality_gap = 5296.23
Duality_gap / Duality_gap_init = 0.170133
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 37
Primal_func_value = 1.20877e+06
Dual_func_value = -1.20238e+06
Duality_gap = 6389.41
Duality_gap / Duality_gap_init = 0.20525
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 38
Primal_func_value = 1.20796e+06
Dual_func_value = -1.20246e+06
Duality_gap = 5499.85
Duality_gap / Duality_gap_init = 0.176674
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 39
Primal_func_value = 1.20895e+06
Dual_func_value = -1.20253e+06
Duality_gap = 6419.42
Duality_gap / Duality_gap_init = 0.206214
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.81469726

Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=0.029369354248046875

Iterations number: 72
Primal_func_value = 1.20227e+06
Dual_func_value = -1.20435e+06
Duality_gap = -2083.56
Duality_gap / Duality_gap_init = -0.0669312
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=3.814697265625e-06

Iterations number: 73
Primal_func_value = 1.20609e+06
Dual_func_value = -1.20435e+06
Duality_gap = 1742.07
Duality_gap / Duality_gap_init = 0.0559612
Binary search results: abs(b-a)=7.62939453125e-06, it=17, beta=0.6327400207519531


ValueError: Couldn't solve quadratic equation. D < 0

### experiments for Beckman model end here.